# Installation


In [ ]:
!pip install git+https://github.com/sberbank-ai/Real-ESRGAN.git

  Cloning https://github.com/sberbank-ai/Real-ESRGAN.git to /tmp/pip-req-build-bfz14z_0
  Running command git clone --filter=blob:none --quiet https://github.com/sberbank-ai/Real-ESRGAN.git /tmp/pip-req-build-bfz14z_0
  Resolved https://github.com/sberbank-ai/Real-ESRGAN.git to commit 362a0316878f41dbdfbb23657b450c3353de5acf
  Preparing metadata (setup.py) ... done


In [ ]:
import os
import cv2
import torch
import shutil
import datetime
import numpy as np
from math import log10, sqrt
from google.colab import files
from RealESRGAN import RealESRGAN
from matplotlib import pyplot as plt

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device:', device)

model_scale2 = "2" #@param ["2", "4", "8"] {allow-input: false}
model_scale4 = "4"
model_scale8 = "8"

model2 = RealESRGAN(device, scale=int(model_scale2))
model2.load_weights(f'weights/RealESRGAN_x{model_scale2}.pth')

model4 = RealESRGAN(device, scale=int(model_scale4))
model4.load_weights(f'weights/RealESRGAN_x{model_scale4}.pth')

model8 = RealESRGAN(device, scale=int(model_scale8))
model8.load_weights(f'weights/RealESRGAN_x{model_scale8}.pth')

device: cuda


#Upload the Video here:

In [ ]:
uploadFolder = 'upload'

if os.path.isdir(uploadFolder):
    shutil.rmtree(uploadFolder)

os.mkdir(uploadFolder)

# Upload video
uploaded = files.upload()
for filename in uploaded.keys():
  destinationPath = os.path.join(uploadFolder, filename)
  print(f'move {filename} to {destinationPath}')
  shutil.move(filename, destinationPath)

Saving WhatsApp Video 2023-09-30 at 12.40.20 (1).mp4 to WhatsApp Video 2023-09-30 at 12.40.20 (1).mp4
move WhatsApp Video 2023-09-30 at 12.40.20 (1).mp4 to upload/WhatsApp Video 2023-09-30 at 12.40.20 (1).mp4


In [ ]:
data = cv2.VideoCapture('/content/upload/' + os.listdir('/content/upload')[0])

frames = data.get(cv2.CAP_PROP_FRAME_COUNT)

# Fetching the number of frames per second
fps = data.get(cv2.CAP_PROP_FPS)
print('Frames per second =',fps)

# Calculating the duration of the video
seconds = round(frames / fps)
videoTime = datetime.timedelta(seconds=seconds)
print(f"Video duration: {videoTime}")

# Counting the total number of frames
totalFrames = int(data.get(cv2.CAP_PROP_FRAME_COUNT))
print("Total number of frames: ", totalFrames)

# Creating folder to save the frame images
resultFolder = 'results'
os.makedirs(resultFolder, exist_ok=True)

Frames per second = 29.97002997002997
Video duration: 0:00:06
Total number of frames:  172


In [ ]:
def isBlurry(image):
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Calculate the Laplacian variance
    laplacianVariance = cv2.Laplacian(gray, cv2.CV_64F).var()

    # If the variance is below the threshold, the image is considered blurry
    return laplacianVariance

def findThreshold(video):
  # Setting the Laplacian Threshold to 50 (after several hit and trials)
  laplacianThreshold = 50
  i = 0
  flag = False
  while True:
      ret, frame = video.read()
      i += 1
      if not ret:
          break

      if isBlurry(frame) < laplacianThreshold:
          print("Pixelated Frame Detected!")
          frameNumber = i
          flag = True
          return frameNumber
      else:
          continue
  if not flag:
    return 1

In [ ]:
thresholdFrame1 = findThreshold(data)
difference = totalFrames - thresholdFrame1
division = int(difference / 3)
thresholdFrame2 = thresholdFrame1 + division
thresholdFrame3 = thresholdFrame2 + division

print("Threshold Frame 1: ", thresholdFrame1)
print("Threshold Frame 2: ", thresholdFrame2)
print("Threshold Frame 3: ", thresholdFrame3)

# Saving the initial non-blurred frames without enhancement
for i in range(0, thresholdFrame1):
  data.set(cv2.CAP_PROP_POS_FRAMES, i)
  ret, frame = data.read()
  result_image_path = os.path.join('results/frame_' + str(i) + '.jpg')
  cv2.imwrite(result_image_path, frame)

# Upscaling the first set of blurry frames by 2x and saving them
for i in range(thresholdFrame1, thresholdFrame2):
  data.set(cv2.CAP_PROP_POS_FRAMES, i)
  ret, frame = data.read()
  sr_image = model2.predict(np.array(frame))
  result_image_path = os.path.join('results/frame_' + str(i) + '.jpg')
  sr_image.save(result_image_path)

# Upscaling the second set of blurry frames by 4x and saving them
for i in range(thresholdFrame2, thresholdFrame3):
  data.set(cv2.CAP_PROP_POS_FRAMES, i)
  ret, frame = data.read()
  sr_image = model4.predict(np.array(frame))
  result_image_path = os.path.join('results/frame_' + str(i) + '.jpg')
  sr_image.save(result_image_path)

# Upscaling the third set of blurry frames by 8x and saving them
for i in range(thresholdFrame3, totalFrames):
  data.set(cv2.CAP_PROP_POS_FRAMES, i)
  ret, frame = data.read()
  sr_image = model8.predict(np.array(frame))
  result_image_path = os.path.join('results/frame_' + str(i) + '.jpg')
  sr_image.save(result_image_path)

Threshold Frame 1:  1
Threshold Frame 2:  58
Threshold Frame 3:  115


In [ ]:
imgArray = []

# Fetching the size of an image frame in order to have a common size for the complete video
sample = cv2.imread(os.path.join('results/frame_' + str(thresholdFrame1 + 1) + '.jpg'))
height, width, channels = sample.shape

for i in range(0, totalFrames):
  # Reading image frames
  img = cv2.imread(os.path.join('results/frame_' + str(i) + '.jpg'))

  # Converting the enhanced frames to RGB
  if i > thresholdFrame1 - 1:
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

  img = cv2.resize(img, (width, height))
  size = (width, height)
  imgArray.append(img)

# Setting 4-character-code
fourcc = cv2.VideoWriter_fourcc(*'DIVX')

out = cv2.VideoWriter('OutputVideo.mp4', fourcc, int(fps), size)

# Writing the frames into the output video file
for i in range(len(imgArray)):
  out.write(imgArray[i])
out.release()